# View submissions for assignment

In [ ]:
# Use nbgrader api to verify that users have been imported into nbgrader
import os
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from datetime import datetime

# Use python-dotenv to load environment variables for better security
from dotenv import load_dotenv
load_dotenv()

COURSE_ROOT = os.environ.get("course-root")
COURSE_ID = os.environ.get("course-id")
HOME = os.environ.get("HOME")

# create a custom config object to specify options for nbgrader API
config = Config()
config.CourseDirectory.root = COURSE_ROOT
config.CourseDirectory.course_id = COURSE_ID

api = NbGraderAPI(config=config)

In [ ]:
from datetime import datetime
import functools

def echo(func):
    '''Decorator to echo (print) lines that are given to func
    (usually an IOTextWrapper)
    '''
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(*args[1:],end='') # pop self from args
        result = func(*args, **kwargs)
        return result
    return wrapper

def utc_to_localtime(timestamp, **kwargs):
    import pytz
    
    inputformat = kwargs.get('inputformat','%Y-%m-%dT%H:%M:%S.%f')
    outputformat = kwargs.get('outputformat','%Y-%m-%d %H:%M')
    
    if type(timestamp) == datetime:
        dt = timestamp
    else:
        try:
            dt = datetime.strptime(timestamp, inputformat)
        except ValueError:
            dt = datetime.strptime(timestamp, inputformat.replace('.%f',''))
    localtime = dt.replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Asia/Singapore'))
    new_timestamp = localtime.strftime(outputformat)
    return new_timestamp


class CSVData:
    '''Wrapper class for list of strings to be written to a CSV file'''
    def __init__(self):
        self.data = []
    
    @echo
    def append(self,line):
        if not line.endswith('\n'):
            line += '\n'
        self.data.append(line)

    def export(self,filename, **kwargs):
        mode = kwargs.get('mode','w')
        with open(filename,'w') as f:
            f.writelines(self.data)
        print(f'Data saved to {filename}')
        
# Collect all submissions
released = list(api.get_released_assignments())
for assignment in released:
    api.collect(assignment)

In [ ]:
data = CSVData()
#Write header
localtime = utc_to_localtime(datetime.now())
line = f'{localtime},{",".join(released)}\n'
data.append(line)
#Write lines
for this in [student['id'] for student in api.get_students()]:
    # Dirty hack to ignore zombie users
    if this.replace('jupyter-','') in ignore:
        continue
    # Build lists of student's submissions and timestamps
    submitted = api.get_student_submissions(this)
    assignments = [s['name'] for s in submitted]
    timestamps = [s['timestamp'] if s['timestamp'] is not None else '' for s in submitted]
    #Write data
    name = this.replace('jupyter-','')
    line = f'{name},' # Start a new line
    for each in released:
        idx = assignments.index(each)
        submission_time = timestamps[idx]
        if each in assignments:
            if submission_time != '':
                localtime = utc_to_localtime(timestamps[idx])
            else:
                localtime = ''
            if each != released[-1]: # not last element
                line += f'{localtime},'
            else:
                line += f'{localtime}\n'
    data.append(line)
data.export(f'{HOME}/admin/submissions.csv')

In [ ]:
%run $HOME/admin/sync-all-to-dropbox.ipynb